<a href="https://colab.research.google.com/github/KarthiksShow/FinetuneLLM/blob/main/Fine_Tune_LLM_Part_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install datasets pandas torch transformers[torch] python-dotenv peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 83.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()  # Load the .env file
hf_token = os.getenv("HF_TOKEN")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login
login(token=hf_token)
model_name = "distilgpt2"
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Set pad token
tokenizer.padding_side = "right"
model = AutoModelForCausalLM.from_pretrained(model_name)
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
text = "ஒரு நாள் "
inputs = tokenizer(text, return_tensors="pt")
# Generate story
output = model.generate(inputs.input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


ஒரு நாள் நாள் நாள் நாள் நாள் நாள் நாள் நாள் நாள


In [ ]:
from datasets import load_dataset
# Load English-to-Colloquial Tamil dataset
raw_data = load_dataset("tniranjan/aitamilnadu_tamil_stories_no_instruct", split="train[:1000]")
data = raw_data.train_test_split(train_size=0.95)
data

README.md:   0%|          | 0.00/628 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/4.34M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/434k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1081 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/121 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 950
    })
    test: Dataset({
        features: ['text'],
        num_rows: 50
    })
})

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
def preprocess_batch(batch):
    return tokenizer(batch["text"], truncation=True, padding=True, max_length=200)
tokenized_dataset = data.map(
preprocess_batch,
batched=True,
batch_size =4,
remove_columns=data["train"].column_names,)
# Print dataset details
print(tokenized_dataset)

Map:   0%|          | 0/950 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 950
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 50
    })
})


In [ ]:
#Data collator
from transformers import DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)
data_collator

DataCollatorForLanguageModeling(tokenizer=GPT2TokenizerFast(name_or_path='distilgpt2', vocab_size=50257, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False, added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}
), mlm=False, mlm_probability=0.15, mask_replace_prob=0.8, random_replace_prob=0.1, pad_to_multiple_of=None, tf_experimental_compile=False, return_tensors='pt', seed=None)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
# Define LoRA Configuration
lora_config = LoraConfig(
    r=8,  # Rank of LoRA matrices (adjust for speed/memory tradeoff)
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout for stability
    bias="none",  # No extra bias parameters
    task_type=TaskType.CAUSAL_LM  # Since we're fine-tuning GPT-style models
)
# Apply LoRA to the model
model = get_peft_model(model, lora_config)
# Print trainable parameters
model.print_trainable_parameters()

trainable params: 147,456 || all params: 82,060,032 || trainable%: 0.1797


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1264: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
model.train() #This ensures layers like Dropout & BatchNorm are active during training.
from torch.optim import AdamW  # Correct import
# Define new optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="./output",
evaluation_strategy="epoch",
#save_strategy="epoch",
save_steps=500,
learning_rate=1e-5,
weight_decay=0.01,
num_train_epochs=3,
per_device_train_batch_size=2,    # Batch size (adjust for GPU memory)
per_device_eval_batch_size=2,
logging_steps=50,
logging_dir="./logs",
resume_from_checkpoint=True
)
trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    optimizers=(optimizer, None),
    data_collator=data_collator
)
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Epoch,Training Loss,Validation Loss
1,1.223400,1.138371
2,1.189500,1.116762
3,1.197500,1.111088


TrainOutput(global_step=1425, training_loss=1.2104086651718406, metrics={'train_runtime': 81.3728, 'train_samples_per_second': 35.024, 'train_steps_per_second': 17.512, 'total_flos': 145952686080000.0, 'train_loss': 1.2104086651718406, 'epoch': 3.0})

In [ ]:
model = AutoModelForCausalLM.from_pretrained ("/content/drive/My Drive/fine_tuned_distilgpt2_Tamil")
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): lora.Linear(
            (base_layer): Conv1D(nf=2304, nx=768)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False

In [ ]:
text = "ஒரு நாள்"
inputs = tokenizer(text, return_tensors="pt")
# Generate story
output = model.generate(inputs.input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ஒரு நாள்ரு நாிரு நாிரு நாிரு நாிரு நாிரு நாிர�


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
trainer.save_model("/content/drive/My Drive/fine_tuned_distilgpt2_Tamil")
tokenizer.save_pretrained("/content/drive/My Drive/fine_tuned_distilgpt2_Tamil")


('/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/tokenizer_config.json',
 '/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/special_tokens_map.json',
 '/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/vocab.json',
 '/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/merges.txt',
 '/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/added_tokens.json',
 '/content/drive/My Drive/fine_tuned_distilgpt2_Tamil/tokenizer.json')

In [ ]:
model = AutoModelForCausalLM.from_pretrained ("/content/drive/My Drive/fine_tuned_distilgpt2_200")
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [ ]:
text = "ஒரு நாள் "
inputs = tokenizer(text, return_tensors="pt")
# Generate story
output = model.generate(inputs.input_ids, max_new_tokens=100)
print(tokenizer.decode(output[0], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


ஒரு நாள் முல்லா ஒரு துணிக் கடைக்குச் சென்றார்


In [ ]:
model.train() #This ensures layers like Dropout & BatchNorm are active during training.
from torch.optim import AdamW  # Correct import
# Define new optimizer
optimizer = AdamW(model.parameters(), lr=1e-5)
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
output_dir="./output",
evaluation_strategy="epoch",
#save_strategy="epoch",
save_steps=500,
learning_rate=1e-5,
weight_decay=0.01,
num_train_epochs=3,
per_device_train_batch_size=2,    # Batch size (adjust for GPU memory)
per_device_eval_batch_size=2,
logging_steps=50,
logging_dir="./logs",
resume_from_checkpoint=True
)
trainer = Trainer(
    model = model,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    args=training_args,
    optimizers=(optimizer, None),
    data_collator=data_collator
)
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.379000,0.186699
2,0.339500,0.184116
3,0.317300,0.179314


TrainOutput(global_step=1425, training_loss=0.3473756462231017, metrics={'train_runtime': 153.1652, 'train_samples_per_second': 18.607, 'train_steps_per_second': 9.304, 'total_flos': 145448386560000.0, 'train_loss': 0.3473756462231017, 'epoch': 3.0})